<a href="https://colab.research.google.com/github/Vadim-Kolesnikov/Neuro-manager/blob/main/neuro_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Установка необходимых зависимостей

In [ ]:
!pip install openai tiktoken langchain langchain-openai langchain-community chromadb

## Импорт библиотек

In [2]:
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from openai import OpenAI
import tiktoken
import re
import os
import requests
import getpass

## Общие переменные

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
MODEL_NAME = "gpt-3.5-turbo"
MODEL = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ссылка на плохо структурированный документ
DOC_URL = "https://docs.google.com/document/d/1ami_prVapm4JeINas-LK_PI4KrALOwSZlDFhziLN5Cs/export?format=txt"

## Структурирование текста

In [4]:
final_text = ""
start_text = requests.get(DOC_URL).text

# загружаем куски текста в модель и просим GPT их структурировать
# после чего структурированный кусок добавляю к остальному тексту
# и так пока текст не закончится
while len(start_text) != 0:
  query1 = f"я отправлю тебе текст ты его структурируешь и отправишь обратно. вот текст: {start_text[:3000]}"
  messages = [
                {"role": "system", "content": ""},
                {"role": "user", "content": query1}
            ]
  completion = MODEL.chat.completions.create(
               model=MODEL_NAME,
               messages=messages,
               temperature=1)

  final_text += completion.choices[0].message.content

  start_text = start_text[3000:]

## Вспомогательный класс

In [ ]:
# служебная информация для модели
models_info = [
    {
        "model_name": "Нейро-менеджер поддержки начинающих преподавателей в онлайн школе",
        "fit_doc": final_text,
        "query_example": "Что делать если ученик не пришел на урок?",
        "role": '''Ты менеджер поддержки преподавателей в онлайн школе программирования под названием CompanyName,
                   к тебе могут обращаться начинающие преподаватели за подсказками и ответами на их вопросы в чате компании.
                   Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у преподавателя не осталось больше вопросов к тебе.
                   Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация.
                   Отвечай максимально точно по документу, не придумывай ничего от себя.
                   Документ с информацией для ответа преподавателю: '''
    }
]


# класс для более комфортного взаимодействия с моделью
class Employee:
  def __init__(self):
    self.model_name = MODEL_NAME
    self.model = MODEL
    self.search_base = None
    self.log = []

  # логирование
  def add_log(self, log_text):
    self.log.append(log_text)

  # функция для создания векторной базы данных
  def create_search_base(self, text):
        # разбиваем текст на чанки
        source_chunks = []
        splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

        for chunk in splitter.split_text(text):
            source_chunks.append(Document(page_content=chunk, metadata={}))

        # формируем из чанков векторную базу данных используя эмбендинги OpenAi
        self.search_base = Chroma.from_documents(source_chunks, OpenAIEmbeddings())
        self.add_log('Данные из документа загружены в в векторную базу данных')

        return self.search_base

  # генераия ответа
  def get_answer(self, system, topic, temp = 100):
        temp = temp / 100
        if not self.search_base:
            self.add_log('Модель необходимо обучить!')
            return ''

        # ищем в базе максимально похожий на вопрос раздел
        docs = self.search_base.similarity_search(topic, k=5)
        message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'Отрывок документа №{i+1}:\n' + doc.page_content + '\\n' for i, doc in enumerate(docs)]))

        # найденный раздел передаем модели в качестве опорного документа
        messages = [
            {"role": "system", "content": system + f"{message_content}"},
            {"role": "user", "content": topic}
        ]

        # формируем запрос
        completion = self.model.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=temp
        )

        # логи
        self.add_log(f'Токенов использовано всего (вопрос): {completion.usage.prompt_tokens}')
        self.add_log(f'Токенов использовано всего (вопрос-ответ): {completion.usage.total_tokens}')

        # возвращаем ответ
        return completion.choices[0].message.content

# создание экземпляра класса и базы знаний для него
employee = Employee()
employee.create_search_base(models_info[0]["fit_doc"])

## Пример использования системы

In [17]:
# вспомогательная функция для получения ответа
def get_answer(quest):
  ans = employee.get_answer(
      models_info[0]["role"], quest
      )
  return ans, employee.log

# вопросы
queries = ['Что делать если ученик не пришел на урок?', 'Как заполнять аналитику после урока?', 'Как получить выплату?']

# вывод ответов на вопросы
for query in queries:
  print('-----------------------------------------')
  print(f'Вопрос: {query}')
  ans, log = get_answer(query)
  print(f'Ответ: {ans}')
  print(f'Логи: {log[-1]}')
  print('-----------------------------------------')

-----------------------------------------
Вопрос: Что делать если ученик не пришел на урок?
Ответ: Если ученик не пришел на урок, нужно дождаться 15 минут после начала занятия. Если ученик так и не подключился, тогда нужно заполнить аналитику соответствующим образом (например, Гугл диск --> Папка "примеры и шаблоны").
Логи: Токенов использовано всего (вопрос-ответ): 2597
-----------------------------------------
-----------------------------------------
Вопрос: Как заполнять аналитику после урока?
Ответ: После каждого урока необходимо создать новую запись в аналитике посещаемости для каждого наставника. Если студент был присутствует, просто заполняем информацию о посещении урока. Если студента не было, также создаем запись и отмечаем отсутствие. Если возникли форс-мажорные обстоятельства, заполняем аналитику согласно примерам из видео на общем диске.
Логи: Токенов использовано всего (вопрос-ответ): 2067
-----------------------------------------
-----------------------------------------